In [201]:
import sys
import pandas as pd
import numpy as np

train_data_path = '../../../data/hw1/train.csv'
test_data_path = '../../../data/hw1/test.csv'


data = pd.read_csv(train_data_path)

# **Preprocessing** 
取需要的數值部分，从第3列开始截取，將 'RAINFALL' 字段的'NR'替换为0

In [202]:
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()

# **Extract Features (1)**
將原始 4320 * 24 的資料依照每個月分重組成 12 個 18 (features) * 480 (hours) 的資料.  
下图为一个月数据的转换，把每个月数据转好，放到month_data[12]中
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/2bcba0f134310b17fa4fde18e68f5cf5.jpg)

In [203]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample

# **Extract Features (2)**
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p5.png)
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p6.png)

每個月會有 480hrs，每 10 小時形成一组 data 与 answer（第10个小时为一个answer）  
每個月會有 471 组 data（第1组：[1 2..10]  第2组[2 3..11] .. 第471组[471 472..480]）  
故總資料數為 471 * 12 组，而每组中 data 有 9 * 18 个features (一小時 18 個 features * 9 小時)，
target 則也有18個(第 10 個小時的 PM2.5)

In [204]:
#x: 471组/月 * 12月  ,18个特征*9小时
sample_x = np.empty([12 * 471, 18 * 9], dtype = float)
#y: 471组/月 * 12月, 1个结果(pm2.5)
sample_y = np.empty([12 * 471, 1], dtype = float)

for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
                
            # month_data[month] => 18x480   (18x9).reshape(1,-1) => (1x162)
            sample_x[month * 471 + day * 24 + hour, :] = \
                month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            
            # month_data[month][9, day * 24 + hour + 9]  9表示第10行为pm2.5，并取第10小时的值
            sample_y[month * 471 + day * 24 + hour, 0] = \
                month_data[month][9, day * 24 + hour + 9] #value
print(sample_x)
print(sample_y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


# **Normalize (1)**
这里使用的是z-score标准化（标准差标准化）  
为什么要标准化数据？  
引用：https://blog.csdn.net/weixin_44700798/article/details/110914533
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/微信截图_20210127134611.png)


In [205]:
#x:[12 * 471, 18 * 9] axis=0 同列相加求平均 结果数=列数
mean_x = np.mean(sample_x, axis = 0) 
#x:[12 * 471, 18 * 9] axis=0 算同列数据 标准差， 结果数=列数
std_x = np.std(sample_x, axis = 0) #18 * 9 

for i in range(len(sample_x)): #12 * 471
    for j in range(len(sample_x[0])): #18 * 9 
        if std_x[j] != 0:
            sample_x[i][j] = (sample_x[i][j] - mean_x[j]) / std_x[j]
sample_x

array([[-1.35825331, -1.35883937, -1.359222  , ...,  0.26650729,
         0.2656797 , -1.14082131],
       [-1.35825331, -1.35883937, -1.51819928, ...,  0.26650729,
        -1.13963133, -1.32832904],
       [-1.35825331, -1.51789368, -1.67717656, ..., -1.13923451,
        -1.32700613, -0.85955971],
       ...,
       [-0.88092053, -0.72262212, -0.56433559, ..., -0.57693779,
        -0.29644471, -0.39079039],
       [-0.7218096 , -0.56356781, -0.72331287, ..., -0.29578943,
        -0.39013211, -0.1095288 ],
       [-0.56269867, -0.72262212, -0.88229015, ..., -0.38950555,
        -0.10906991,  0.07797893]])

#**Split Training Data Into "train_set" and "validation_set"**  
80% 作为训练集 train_set
20% 作为验证集 validation_set

In [206]:
import math
# math.floor()向下取整
x_train_set = sample_x[: math.floor(len(sample_x) * 0.8), :]
y_train_set = sample_y[: math.floor(len(sample_y) * 0.8), :]
x_validation = sample_x[math.floor(len(sample_x) * 0.8): , :]
y_validation = sample_y[math.floor(len(sample_y) * 0.8): , :]
# print(x_train_set)
# print(y_train_set)
# print(x_validation)
# print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

4521
4521
1131
1131


# **Training**
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p8.png)
(和上圖不同處: 下面的 code 採用 Root Mean Square Error)

因為常數項的存在，所以 dimension (dim) 需要多加一欄；eps 項是避免 adagrad 的分母為 0 而加的極小數值。

每一個 dimension (dim) 會對應到各自的 gradient, weight (w)，透過一次次的 iteration (iter_time) 學習。

![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/微信截图_20210127185035.png)
上面的loss函数J(θ)在代码中使用矩阵时，计算平方和可以这样
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/微信截图_20210128105105.png)
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p9.png)
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p10.png)


In [207]:
#18*9个θ  1个b
dim = 18 * 9 + 1

w = np.zeros([dim, 1])
m = 12 * 471
#x 拼接1列值为1，用来与常数b相乘
x = np.concatenate((np.ones([m, 1]), sample_x), axis = 1).astype(float)
y = sample_y

learning_rate = 100 #学习率
iter_time = 1000 #训练次数

adagrad = np.zeros([dim, 1])
gradient = np.zeros([dim, 1])
eps = 0.0000000001 #小量 避免 adagrad 的分母為 0 而加的極小數值
for t in range(iter_time):
    # ( y'_vector - y_vector )^2
    y_diff_square = np.power(np.dot(x, w) - y, 2)
    
    # 1/n * (Σ (y' - y)^2)
    mean_y_diff_square = np.sum(y_diff_square)/m/2
    
    # 开根号
    # loss = np.sqrt(mean_y_diff_square)#rmse
    loss = mean_y_diff_square
   
    gradient = np.dot(x.T, np.dot(x, w) - y)/m #dim*1
    adagrad += gradient ** 2
    w = w -  learning_rate  * gradient / np.sqrt(adagrad + eps)
    
    if t % 100 == 0:
        print(str(t) + " loss:"+ "{:.2f}".format(loss))

print(w)
np.save('weight.npy', w)

0 loss:366.43
100 loss:570.85
200 loss:198.28
300 loss:91.54
400 loss:56.66
500 loss:43.03
600 loss:36.28
700 loss:32.11
800 loss:29.16
900 loss:26.91
[[ 2.13740269e+01]
 [ 3.58888910e+00]
 [ 4.56386305e+00]
 [ 2.16307009e+00]
 [-6.58545205e+00]
 [-3.38885577e+01]
 [ 3.22235513e+01]
 [ 3.49340411e+00]
 [-4.60308473e+00]
 [-1.02374968e+00]
 [-3.96791485e-01]
 [-1.06908798e-01]
 [ 2.22488179e-01]
 [ 8.99634218e-02]
 [ 1.31243107e-01]
 [ 2.15894870e-02]
 [-1.52867265e-01]
 [ 4.54087634e-02]
 [ 5.20999246e-01]
 [ 1.60824220e-01]
 [-3.17709422e-02]
 [ 1.28528976e-02]
 [-1.76839441e-01]
 [ 1.71241370e-01]
 [-1.31190028e-01]
 [-3.51614441e-02]
 [ 1.00826192e-01]
 [ 3.45018256e-01]
 [ 4.00130204e-02]
 [ 2.54331640e-02]
 [-5.04425224e-01]
 [ 3.71483000e-01]
 [ 8.46357674e-01]
 [-8.11920397e-01]
 [-8.00217811e-02]
 [ 1.52737675e-01]
 [ 2.64915166e-01]
 [-5.19860382e-02]
 [-2.51988304e-01]
 [ 3.85246501e-01]
 [ 1.65431467e-01]
 [-7.83633230e-02]
 [-2.89457239e-01]
 [ 1.77615009e-01]
 [ 3.22506948

# **Testing**
![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p11.png)  
只要求test 240笔数据  
載入 test data，並且以相似於訓練資料預先處理和特徵萃取的方式處理，使 test data 形成 240 個維度為 18 * 9 + 1 的資料。

In [208]:
# testdata = pd.read_csv('gdrive/My Drive/hw1-regression/test.csv', header = None, encoding = 'big5')
testdata = pd.read_csv(test_data_path, header = None)
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            #这里标准化，使用的是样本的均值和标准差
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

<ipython-input-208-8b6c5931c39f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[test_data == 'NR'] = 0
D:\Program Files\Anaconda3\envs\study\lib\site-packages\pandas\core\frame.py:3212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


array([[ 1.        , -0.24447681, -0.24545919, ..., -0.67065391,
        -1.04594393,  0.07797893],
       [ 1.        , -1.35825331, -1.51789368, ...,  0.17279117,
        -0.10906991, -0.48454426],
       [ 1.        ,  1.5057434 ,  1.34508393, ..., -1.32666675,
        -1.04594393, -0.57829812],
       ...,
       [ 1.        ,  0.3919669 ,  0.54981237, ...,  0.26650729,
        -0.20275731,  1.20302531],
       [ 1.        , -1.8355861 , -1.8360023 , ..., -1.04551839,
        -1.13963133, -1.14082131],
       [ 1.        , -1.35825331, -1.35883937, ...,  2.98427476,
         3.26367657,  1.76554849]])

# **Prediction**
說明圖同上

![](https://picbed-xunxun.oss-cn-shanghai.aliyuncs.com/0217p11.png)

有了 weight 和測試資料即可預測 target。

In [209]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
# ans_y

# **Save Prediction to CSV File**


In [210]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 5.174960617497138]
['id_1', 18.306214158579632]
['id_2', 20.491218266099292]
['id_3', 11.52394311945313]
['id_4', 26.616056725302517]
['id_5', 20.53134803059195]
['id_6', 21.906551020090273]
['id_7', 31.73646862816737]
['id_8', 13.391674308046067]
['id_9', 64.45646589573622]
['id_10', 20.26456868765696]
['id_11', 15.35857609108095]
['id_12', 68.5894728475939]
['id_13', 48.428113552710904]
['id_14', 18.702333608380457]
['id_15', 10.188595749801511]
['id_16', 30.740363104187967]
['id_17', 71.13221771540199]
['id_18', -4.130517432817689]
['id_19', 18.2356939190336]
['id_20', 38.57892224542915]
['id_21', 71.31151971433806]
['id_22', 7.410348293187002]
['id_23', 18.717955303948685]
['id_24', 14.937250167918082]
['id_25', 36.71973664117148]
['id_26', 17.961696886681136]
['id_27', 75.78946265598819]
['id_28', 12.309310468451772]
['id_29', 56.2953514038601]
['id_30', 25.113160895161325]
['id_31', 4.610248691072931]
['id_32', 2.4837708720617773]
['id_33', 24.75942217873

相關 reference 可以參考:

Adagrad :
https://youtu.be/yKKNr-QKz2Q?list=PLJV_el3uVTsPy9oCRY30oBPNLCo89yu49&t=705 

RMSprop : 
https://www.youtube.com/watch?v=5Yt-obwvMHI 

Adam
https://www.youtube.com/watch?v=JXQT_vxqwIs 


以上 print 的部分主要是為了看一下資料和結果的呈現，拿掉也無妨。另外，在自己的 linux 系統，可以將檔案寫死的的部分換成 sys.argv 的使用 (可在 terminal 自行輸入檔案和檔案位置)。

最後，可以藉由調整 learning rate、iter_time (iteration 次數)、取用 features 的多寡(取幾個小時，取哪些特徵欄位)，甚至是不同的 model 來超越 baseline。

Report 的問題模板請參照 : https://docs.google.com/document/d/1s84RXs2AEgZr54WCK9IgZrfTF-6B1td-AlKR9oqYa4g/edit